In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/Jakob/Desktop/Dataset/Air_temperature/January.txt', sep=";", header=1)

In [2]:
df

,Jahr,Monat,Brandenburg/Berlin,Brandenburg,Baden-Wuerttemberg,Bayern,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Niedersachsen/Hamburg/Bremen,Nordrhein-Westfalen,Rheinland-Pfalz,Schleswig-Holstein,Saarland,Sachsen,Sachsen-Anhalt,Thueringen/Sachsen-Anhalt,Thueringen,Deutschland,Unnamed: 19
0,1881,1,-5.54,-5.56,-4.89,-6.51,-5.68,-5.07,-4.55,-4.55,-4.21,-4.49,-4.06,-4.15,-6.22,-5.89,-6.28,-6.76,-5.36,NaN
1,1882,1,1.43,1.42,-0.65,-1.33,-0.30,1.59,1.73,1.73,1.26,-0.17,2.11,-0.04,0.63,1.18,0.60,-0.12,0.41,NaN
2,1883,1,-0.41,-0.42,-0.09,-1.49,0.04,-0.69,0.33,0.32,1.27,0.49,-0.12,0.55,-1.20,-0.39,-0.69,-1.07,-0.32,NaN
3,1884,1,3.35,3.34,2.34,1.22,2.97,3.16,3.99,3.99,4.17,3.24,3.56,3.24,2.39,3.32,2.93,2.43,2.86,NaN
4,1885,1,-2.15,-2.16,-3.88,-4.91,-2.66,-1.71,-1.44,-1.44,-1.14,-2.01,-1.09,-2.00,-2.87,-2.86,-3.36,-3.98,-2.79,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2019,1,1.18,1.17,-0.32,-0.95,0.50,1.51,1.99,2.00,1.64,0.83,2.12,0.98,-0.20,1.23,0.59,-0.24,0.63,NaN
139,2020,1,4.10,4.09,2.51,1.36,3.22,4.66,4.99,5.00,4.69,3.64,5.43,3.74,2.98,4.18,3.57,2.77,3.48,NaN
140,2021,1,0.74,0.72,0.21,-0.74,0.75,1.11,1.65,1.66,1.94,1.18,1.66,1.24,-0.19,0.76,0.34,-0.22,0.63,NaN
141,2022,1,3.39,3.38,1.34,1.07,2.66,3.68,4.47,4.48,3.77,2.44,4.69,2.38,2.28,3.73,3.11,2.29,2.79,NaN


Preprocessing Pipeline:

* Drop data before 1996
* Create a new column consisting of the year and month (format: YYYY-MM)
* Rename the Column "Deutschland" to "Temperature (°C)"
* Delete unnecassary columns
* Change the order

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataDropper(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X.drop(X[X["Jahr"]<1996].index)
        X.reset_index(drop=True)
        return X
    
    
class ColumnCreater(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X["Date"] = X["Jahr"].astype(str) + "-" + "0" + df["Monat"].astype(str)
        return X
    
    
class NameChanger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X.rename(columns = {"Deutschland":"Temperature (°C)"}, inplace=True)
        return X
    

class ColumnDropper(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        columns_to_delete = list(range(0,18))
        return X.drop(X.columns[columns_to_delete], axis=1)
    

class OrderChanger(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        new_columns_order = ["Date", "Temperature (°C)"]
        return X[new_columns_order]

In [4]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("ddropper", DataDropper()),
    ("creater", ColumnCreater()),
    ("nchanger", NameChanger()),
    ("cdropper", ColumnDropper()),
    ("ochanger", OrderChanger())
])

pipe.fit_transform(df)

,Date,Temperature (°C)
0,1881-01,-5.36
1,1882-01,0.41
2,1883-01,-0.32
3,1884-01,2.86
4,1885-01,-2.79
...,...,...
138,2019-01,0.63
139,2020-01,3.48
140,2021-01,0.63
141,2022-01,2.79
